In [7]:
import pandas as pd
import numpy as np
from collections import defaultdict
import os
from pprint import pprint

# csv 파일의 저장될 상위 경로
upper_dir = '/opt/p4-dkt-freshtomato/output_ensemble/'
os.makedirs(upper_dir, exist_ok=True)

# 원하는 csv 파일의 이름
filename = 'ensembled_16개_hard_and_soft_ensemble' + '.csv'

filename = f'{upper_dir}/{filename}'

data1 = pd.read_csv('/opt/p4-dkt-freshtomato/CSV_For_Ensemble/output_CNN.csv')
data2 = pd.read_csv('/opt/p4-dkt-freshtomato/CSV_For_Ensemble/output_GRUATTN.csv')
data3 = pd.read_csv('/opt/p4-dkt-freshtomato/CSV_For_Ensemble/output_LSTM.csv')
data4 = pd.read_csv('/opt/p4-dkt-freshtomato/CSV_For_Ensemble/output_ML.csv')

# data_list 에 앙상블을 원하는 dataframe들을 넣어주면 됩니다.
# data_list = [data1, data2, data3,data4]
data_list = [pd.read_csv(f'/opt/p4-dkt-freshtomato/CSV_For_Ensemble/output{num}.csv') for num in range(16)]
# predictions_arr

In [8]:
n_bins = 10
def hard_and_soft_ensemble_by_bins(filename, data_list, n_bins):
    predictions_arr = np.array([data.prediction.values for data in data_list]).T
    from collections import Counter
    n_bins = n_bins
    bins = [b for b in np.arange(0,1,1/n_bins)]
    belong_where_bin = [np.digitize(predictions,bins=bins) for predictions in predictions_arr]
    counter_list = [Counter(b) for b in belong_where_bin]
    # pprint(belong_where_bin[:5])    # 구간 name
    # pprint(counter_list[:5])

    result_list = []
    for belong_bin, counter, pred_vals_in_one_row in zip(belong_where_bin, counter_list, predictions_arr):
        freq_max = max(counter.values())
        # print(freq_max)
        names_bin_have_maxfreq_per_row = np.array([idx for idx,count in counter.items() if count == freq_max])
        # print(names_bin_have_maxfreq_per_row)
        where_max_idxes = [idx for idx, n_bin in enumerate(belong_bin) if n_bin in names_bin_have_maxfreq_per_row]
        # print(where_max_idxes)
        result_list.append(np.mean(pred_vals_in_one_row[where_max_idxes]))

    final_csv=pd.DataFrame([[idx, pred_val] for idx, pred_val in enumerate(result_list)], columns=['id','prediction'])
    final_csv.to_csv(filename, index=False)
    return final_csv

In [9]:
hard_and_soft_ensemble_by_bins(filename=filename, data_list=data_list, n_bins=10)

,id,prediction
0,0,0.734132
1,1,0.831526
2,2,0.204848
3,3,0.733402
4,4,0.151269
...,...,...
739,739,0.042471
740,740,0.826355
741,741,0.741395
742,742,0.843829


In [77]:
# # threshold 하나 기준으로 hard voting 후, soft ensemble

# result_list = []
# for values_list in predictions_arr.T:
#     ths = 0.7
#     close_to_0 = []
#     close_to_1 = []

#     for value in values_list:
#         if value >= ths:
#             close_to_1.append(value)
#         else:
#             close_to_0.append(value)
#     if len(close_to_1)>=len(close_to_0):   # 1에 가까운 value들이 더 많은 경우
#         result = np.mean(close_to_1)
#     else:
#         result = np.mean(close_to_0)
#     result_list.append(result)
# final_csv = pd.DataFrame([[idx, pred_val] for idx, pred_val in enumerate(result_list)], columns=['id','prediction'])
# final_csv.to_csv(filename, index=False)

In [41]:
final_csv

,id,prediction
0,0,0.694100
1,1,0.797033
2,2,0.247533
3,3,0.688367
4,4,0.255933
...,...,...
739,739,0.085500
740,740,0.783667
741,741,0.868867
742,742,0.871400
